<a href="https://colab.research.google.com/github/vss121/exhibition-visitor-tracker/blob/main/5-show-traffic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

목록
1. YOLOV5 + ByteTrack으로 인식 -> 프레임별 모든 id 좌표값 리턴
2. 이동거리 계산 후 클러스터링 -> 이동거리 작은 클러스터 지운 후 남은 id에 대한 프레임별 좌표값 리턴
3. 좌표 시점변환 (탑뷰) -> 변환된 좌표값 리턴
4. 필터링된 ID값만 남은 프레임별 좌표값에 대해 KDE 연산 -> 히트맵 출력(3D로)  
5. 필터링된 ID값만 남은 프레임별 좌표값에 대해 동선 연산 -> 동선 맵 출력(Distinctify)  
6. 필터링된 ID값만 남은 프레임별 좌표값에 대해 가까운 작품 찾는 연산 -> 인기많은 작품 찾기

# 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
result_dir_path = "/content/drive/MyDrive/results"

result_path = "/content/drive/MyDrive/result"
filename = "exhibition-64-full.mp4"

# 로컬 환경에서 연결

In [ ]:
import os
HOME = os.getcwd()
%cd {HOME}

filename = ""

with open(f"{HOME}/filename.txt", 'r') as file:
    for line in file:
        filename = line.strip()

result_path = f"{HOME}/result"

# 5. 필터링된 ID값만 남은 프레임별 좌표값에 대해 동선 연산 -> 동선 맵 출력(Distinctify)  

In [11]:
# distinctipy 설치
!git clone https://github.com/alan-turing-institute/distinctipy.git
%cd distinctipy
!pip install .
%cd ..

fatal: destination path 'distinctipy' already exists and is not an empty directory.
/content/distinctipy/distinctipy
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.
/content/distinctipy


In [12]:
import pandas as pd
import matplotlib.pyplot as plt
from distinctipy import distinctipy

In [13]:
# 파일 불러오기
data = pd.read_csv(f'{result_path}/{filename}-4-trajectories.txt', header=None, names=['id', 'frame', 'x', 'y'])
print(data.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   frame  id  probability    x     y
0      0   1     0.784182  293  1710
1      0   2     0.749597  625  1024
2      0   3     0.728924  574   491
3      0   4     0.690018  544   435
4      0   5     0.680504  459   323


In [14]:
# 데이터 프레임을 그룹화하고 각 그룹에 대해 리스트(변수 result)를 생성
grouped = data.groupby('id')
result = {}
for name, group in grouped:
    result[name] = list(zip(group['frame'], group['x'], group['y']))

In [15]:
width = 400
height = 2000

In [7]:
# id 개수
trajectories_items_num = len(result.items())

# distinctipy 라이브러리 사용
colors = distinctipy.get_colors(trajectories_items_num)
print(colors)

# 색깔 확인
# distinctipy.color_swatch(colors)

# 개수 세기
trajectories_items_num = 0

# 각 객체별로 정렬된 좌표에 따라 경로 그리기
plt.figure(figsize=(2, 10))
for obj_id, path in result.items():
    # 개수 +1
    trajectories_items_num += 1
    # 시간에 따라 경로 정렬
    sorted_path = sorted(path, key=lambda x: x[0])
    # 경로의 x, y 좌표를 분리
    x_coords, y_coords = zip(*[(x, y) for _, x, y in sorted_path])

    # 경로를 그림
    plt.plot(x_coords, y_coords, color=colors[trajectories_items_num - 1], marker='o', label=f'Object {trajectories_items_num}')

# 시각화 설정
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))  # 범례

#plt.imshow(warped_image)
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Tracking data of every visitor')
plt.xlim([0, width])
plt.ylim([height, 0])
plt.grid(True)
plt.axis('equal')  # 축의 스케일을 동일하게 설정

# 결과 표시
plt.show()


KeyboardInterrupt: 